## Links Charges Data Pipeline db way

In [3]:
import os
import pandas as pd
import numpy as np
import glob
from data_cleaner import InvoiceCleaner,SLACleaner

raw_data_folder_path = 'C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\\SLA EDA\\SLA_ETL\\Data'
processed_data_folder_path = 'C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\\SLA EDA\\SLA_ETL\\Processsed_Data'

# Ensuring the processed folder exist
os.makedirs(processed_data_folder_path, exist_ok=True)

excel_files = glob.glob(f"{raw_data_folder_path}/*.xlsx")

# Empty List to be used to append the related dfs
invoice_dfs = []
sla_dfs = []

def sla_invoice(sn):
    if "sla" in sn.lower():
        return "SLA"
    elif "invoice" in sn.lower():
        return "Invoice"
    else:
        return f"Unknown"

for excel_file in excel_files:
    xls = pd.ExcelFile(excel_file)
    workbook_name = os.path.splitext(os.path.basename(excel_file))[0]


    for sheet_name in xls.sheet_names:
        df = pd.read_excel(excel_file, sheet_name=sheet_name)

        # Determine the type of sheet for processing
        sheet_type = sla_invoice(sheet_name)

        # New files Name
        w_b = workbook_name.split(" ")[0]
        new_file_name = f"{w_b}_{sheet_type}.csv"
        new_file_path = os.path.join(processed_data_folder_path, new_file_name)



        # Apply different processing based on sheet type
        if sheet_type == "SLA":

            # Specific processing for SLA sheets            
            df_processed = SLACleaner(df).clean_data()  # Processing the SLA data

            #Adding an SLA_ID Column
            df_processed['SLA_ID'] = f"{w_b}-SLA-{max(df_processed['SLA_Date'])}"

            #Adding the Service Provider Col
            df_processed['Service_Provider'] = w_b

            # Creating a list of SLA Data Processed            
            df_sla = df_processed.copy()
            sla_dfs.append(df_sla)
        elif sheet_type == "Invoice":

            # Specific processing for Invoice sheets            
            df_processed = InvoiceCleaner(df).clean_data()  ## Processing the Invoice data

            #Adding the Service Provider Col
            df_processed['Service_Provider'] = w_b

            # Creating a list of all the Invoice data processed
            df_invoice = df_processed.copy()
            invoice_dfs.append(df_invoice)
        else:
            # Default processing or skip
            df_processed = df  # No processing that take place if the sheet is neither SLA nor Invoice
        

        # Save the processed DataFrame to a new CSV file
        df_processed.to_csv(new_file_path, index=False)
        print(f"Processed and Saved: {new_file_path}")

##Combining the dfs into 1. combined_SLA and 2. combined_invoice
# Appending the SLA and Invoice dfs
SLA_Combined_df = pd.concat(sla_dfs)
Invoice_Combined_df = pd.concat(invoice_dfs)

# Saving them to the Processed Folder
#SLA
SLA_Combined_path = f'{processed_data_folder_path}\\SLA_Combined.csv'
SLA_Combined_df.to_csv(SLA_Combined_path, index= False)
print(f'SLA Combined Saved to{SLA_Combined_path}')

#Invoice
Invoice_Combined_path = f'{processed_data_folder_path}\\Invoice_Combined.csv'
Invoice_Combined_df.to_csv(Invoice_Combined_path, index= False)
print(f'Invoice Combined Saved to{Invoice_Combined_path}')


#### Saving them into a mysql DB
#Importing DBManger class
from db_manager import DB_Manager


#establishing Connection
db_manager = DB_Manager(
    host = "localhost",
    user = "KRA",
    password = "36031445&Mysql_Kra",
    database = "kra_sla_etl_project"
)

#Insert SLA Data Into the DB
for index, row in SLA_Combined_df.iterrows():
    db_manager.insert_data('SLA_Combined_df', row.to_dict())

#Inserting the Invoice data into the db
for index, row in Invoice_Combined_df.iterrows():
    db_manager.insert_data('Invoice_Combined_df', row.to_dict())


print('\n\t Imagine Imerun !!!\t\nNow to Visualization')


Processed and Saved: C:\Users\USER\OneDrive\KRA WORK\2024 WS\SLA EDA\SLA_ETL\Processsed_Data\LTK_SLA.csv
Processed and Saved: C:\Users\USER\OneDrive\KRA WORK\2024 WS\SLA EDA\SLA_ETL\Processsed_Data\LTK_Invoice.csv
Processed and Saved: C:\Users\USER\OneDrive\KRA WORK\2024 WS\SLA EDA\SLA_ETL\Processsed_Data\SAFARICOM_SLA.csv
Processed and Saved: C:\Users\USER\OneDrive\KRA WORK\2024 WS\SLA EDA\SLA_ETL\Processsed_Data\SAFARICOM_Invoice.csv
SLA Combined Saved toC:\Users\USER\OneDrive\KRA WORK\2024 WS\SLA EDA\SLA_ETL\Processsed_Data\SLA_Combined.csv
Invoice Combined Saved toC:\Users\USER\OneDrive\KRA WORK\2024 WS\SLA EDA\SLA_ETL\Processsed_Data\Invoice_Combined.csv
Query: INSERT INTO SLA_Combined_df (Link_ID, SLA_Date, Last_Mile, Capacity(Mbps), Location, MRC_Excl, SLM_Comments, QRC_Incl, SLA_ID, Service_Provider) VALUES (1750, 2020-10-01 00:00:00, 'Fibre', 90, 'KRA - TIMES TOWERS TO KRA - KIXP (EADC)', 270000.0, '16/04/2022: Upgraded 90 Mbps @ KES 649,050  -> 150 Mbps @ KES 810,000 ex vat',

### Testing 

In [1]:
import pandas as pd
ltk_sla_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\LTK_SLA.csv')
saf_sla_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\SAFARICOM_SLA.csv')
combined_sla = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\SLA_Combined.csv')

ltk_invoice_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\LTK_Invoice.csv')
saf_invoice_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\SAFARICOM_Invoice.csv')
combined_Invoice = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\Invoice_Combined.csv')

In [2]:
ltk_invoice_df.head()

,Invoice_Date,Link_ID,Invoice_Period,Invoice_Description,Invoice_Reference,Total_QRC,Invoice_Data,Service_Provider
0,2023-10-06,C-00159-0087,01-Oct-2023 to 31-Dec-2023,C-00159-0087 EPL Connection charge 80MBPS->SAM...,116426,417600.0,2023-10-06,LTK
1,2023-10-06,1750,01-Oct-2023 to 31-Dec-2023,01750 National MPLS KRA - TIMES TOWERS TO VRF,116426,939600.0,2023-10-06,LTK
2,2023-10-06,5611,01-Oct-2023 to 31-Dec-2023,05611 National MPLS Msa link vlan 460 customs,116426,71835.9,2023-10-06,LTK
3,2023-10-06,20103,01-Oct-2023 to 31-Dec-2023,20103 Leased_Circuits_Naticnal KRA WILSON AIRP...,116426,79866.0,2023-10-06,LTK
4,2023-10-06,20954,01-Oct-2023 to 31-Dec-2023,20954 National MPLS KRA EGERTON UNIVERSITY NJO...,116426,174000.0,2023-10-06,LTK


In [4]:
saf_invoice_df.head()

,Invoice_Data,Link_ID,Invoice_Period,Invoice_Description,Invoice_Reference,Total_QRC,Service_Provider
0,2023-10-01,95001135,1st Oct to 31st Dec 2023,CUSTOMS OFFICE MALINDI-WIMAX,B1-10096515502,240145.6128,SAFARICOM
1,2023-10-01,95001574,1st Oct to 31st Dec 2023,KRA LOITOKTOK BORDER WIMAX,B1-10096515502,240145.6128,SAFARICOM
2,2023-10-01,95001628,1st Oct to 31st Dec 2023,KRA KRATI MOMBASA WIMAX,B1-10096515502,240145.6128,SAFARICOM
3,2023-10-01,95004139,1st Oct to 31st Dec 2023,KRA - MANDERA,B1-10096515502,180078.7944,SAFARICOM
4,2023-10-01,95004140,1st Oct to 31st Dec 2023,KRA - Moyale,B1-10096515502,240145.6128,SAFARICOM


In [18]:
combined_sla.head()

,Link_ID,SLA_Date,Last_Mile,Capacity,Location,MRC_Excl,SLM_Comments,QRC_Incl
0,95054463,2020-12-16,MPLS,25,BUNGOMA MICROWAVE,20352.8,Upgrade 5>25 Mbps. Check Upgrade Docs. KES 449...,79986.504
1,95096538,2020-12-16,MPLS,12,BUSIA-WIMAX,70008.3,NaN,275132.619
2,95038953,2020-12-16,MPLS,80,DR BACKHAUL FIBRE,93504.6,NaN,367473.078
3,95054464,2020-12-16,MPLS,60,ELDORET KIPTAGICH MPLS FIBER,25439.4,Upgrade 10>60 Mbps. Check Upgrade Docs. KES 69...,99976.842
4,95054465,2020-12-16,MPLS,5,ELDORET KPC LOOP MICROWAVE,20357.4,NaN,80004.582


In [4]:
combined_sla.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Link_ID           85 non-null     object 
 1   SLA_Date          82 non-null     object 
 2   Last_Mile         85 non-null     object 
 3   Capacity(Mbps)    85 non-null     int64  
 4   Location          83 non-null     object 
 5   MRC_Excl          85 non-null     float64
 6   SLM_Comments      47 non-null     object 
 7   QRC_Incl          85 non-null     float64
 8   SLA_ID            85 non-null     object 
 9   Service_Provider  85 non-null     object 
dtypes: float64(2), int64(1), object(7)
memory usage: 6.8+ KB


In [5]:
combined_Invoice.head()

,Link_ID,SLA_Date,Last_Mile,Capacity(Mbps),Location,MRC_Excl,SLM_Comments,QRC_Incl,SLA_ID,Service_Provider
0,1750,2020-10-01,Fibre,90,KRA - TIMES TOWERS TO KRA - KIXP (EADC),270000.0,"16/04/2022: Upgraded 90 Mbps @ KES 649,050 ->...",939600.00,LTK-SLA-2020-10-01 00:00:00,LTK
1,5277,2020-10-01,Fibre,75,KRA CUSTOMS HSE TO KRA TIMES TOWER,179718.0,Not in Invoice/Renamed/Relocated,625418.64,LTK-SLA-2020-10-01 00:00:00,LTK
2,5611,2020-10-01,Fibre,8,KRA AIRPORT CONTAINER DEPOT TO CUSTOMS HSE MSA,20642.5,Okay. No change,71835.90,LTK-SLA-2020-10-01 00:00:00,LTK
3,20103,2020-10-01,Fibre,9,KRA Wilson Airport Customs to Times Towers - V...,22950.0,Okay. No change,79866.00,LTK-SLA-2020-10-01 00:00:00,LTK
4,20954,2020-10-01,Fibre,20,KRA TIMES TOWERS NBO TO EGERTON UNIVERSITY NJO...,50000.0,Okay. No change,174000.00,LTK-SLA-2020-10-01 00:00:00,LTK


In [5]:
combined_Invoice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Link_ID           85 non-null     object 
 1   SLA_Date          82 non-null     object 
 2   Last_Mile         85 non-null     object 
 3   Capacity(Mbps)    85 non-null     int64  
 4   Location          83 non-null     object 
 5   MRC_Excl          85 non-null     float64
 6   SLM_Comments      47 non-null     object 
 7   QRC_Incl          85 non-null     float64
 8   SLA_ID            85 non-null     object 
 9   Service_Provider  85 non-null     object 
dtypes: float64(2), int64(1), object(7)
memory usage: 6.8+ KB


In [22]:
invoice_dfs


1

In [11]:
DB_Manager(
    host = "localhost",
    user = "KRA",
    password = "36031445&Mysql_Kra",
    database = "kra_sla_etl_project"
).connect()

Connected to database successfully.


In [8]:
import mysql.connector

In [12]:
import mysql.connecto
conn = mysql.connector(
    host = "localhost",
    user = "KRA",
    password = "36031445&Mysql_Kra",
    database = "kra_sla_etl_project"
    
)

TypeError: 'module' object is not callable